### Spark Initialization

In [1]:
from pyspark.sql import SparkSession
from IPython.display import display

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Test PySpark Installation") \
    .getOrCreate()

### Data Reading CSV

In [2]:
df=spark.read.format('csv')\
            .option('inferSchema',True)\
            .option('header',True)\
            .load('BigMart_Sales.csv')

Viewing Data

In [3]:
#it is an Action
df.show() 


+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

### Data Reading Json

In [18]:
df_json=spark.read.format('json')\
                    .option('inferSchema',True)\
                    .option('header',True)\
                    .option('mulltiline',False)\
                    .load('drivers.json')

In [19]:
df_json.show()

+----+----------+--------+----------+--------------------+-----------+------+--------------------+
|code|       dob|driverId| driverRef|                name|nationality|number|                 url|
+----+----------+--------+----------+--------------------+-----------+------+--------------------+
| HAM|1985-01-07|       1|  hamilton|   {Lewis, Hamilton}|    British|    44|http://en.wikiped...|
| HEI|1977-05-10|       2|  heidfeld|    {Nick, Heidfeld}|     German|    \N|http://en.wikiped...|
| ROS|1985-06-27|       3|   rosberg|     {Nico, Rosberg}|     German|     6|http://en.wikiped...|
| ALO|1981-07-29|       4|    alonso|  {Fernando, Alonso}|    Spanish|    14|http://en.wikiped...|
| KOV|1981-10-19|       5|kovalainen|{Heikki, Kovalainen}|    Finnish|    \N|http://en.wikiped...|
| NAK|1985-01-11|       6|  nakajima|  {Kazuki, Nakajima}|   Japanese|    \N|http://en.wikiped...|
| BOU|1979-02-28|       7|  bourdais|{Sébastien, Bourd...|     French|    \N|http://en.wikiped...|
| RAI|1979

### SCHEMA - DDL and StructType()

The below schema is what the schema we get if we infer schema to TRUE

In [43]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



### Schema Definition

In [22]:
my_ddl_schema='''
            Item_Identifier STRING,
            Item_Weight STRING,
            Itemp_Fat_Content STRING,
            Item_Visibility DOUBLE,
            Item_Type STRING,
            Item_MRP DOUBLE,
            Outlet_Identifier STRING,
            Outlet_Establishment_Year INT,
            Outlet_Size String,
            Outlet_Location_Type STRING,
            Outlet_Type STRING,
            Item_Outlet_Sales DOUBLE
            '''

In [24]:
df=spark.read.format('csv')\
                .schema(my_ddl_schema)\
                .option('header',True)\
                .load('BigMart_Sales.csv')

In [27]:
df.show()
df.printSchema()

+---------------+-----------+-----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Itemp_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+-----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|          Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|          Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Su

### StructType() Schema

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [31]:
struct_schema=StructType([
    StructField('Item_Identifier',StringType(),True),
    StructField('Item_Weight',StringType(),True),
    StructField('Itemp_Fat_Content',StringType(),True),
    StructField('Item_Visibility',StringType(),True),
    StructField('Item_Type',StringType(),True),
    StructField('Item_MRP',StringType(),True),
    StructField('Outlet_Identifier',StringType(),True),
    StructField('Outlet_Establishment',StringType(),True),
    StructField('Outlet_Size',StringType(),True),
    StructField('Outlet_Location_Type',StringType(),True),
    StructField('Outlet_Type',StringType(),True),
    StructField('Item_Outlet_Sales',StringType(),True)
])

In [32]:
df=spark.read.format('csv')\
            .schema(struct_schema)\
            .option('Header',True)\
            .load('BigMart_Sales.csv')

In [35]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



## SELECT - To select required columns

In [39]:
df_select=df.select('Item_Identifier','Item_Weight','Item_Fat_Content')
df_select.show()

+---------------+-----------+----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|
+---------------+-----------+----------------+
|          FDA15|        9.3|         Low Fat|
|          DRC01|       5.92|         Regular|
|          FDN15|       17.5|         Low Fat|
|          FDX07|       19.2|         Regular|
|          NCD19|       8.93|         Low Fat|
|          FDP36|     10.395|         Regular|
|          FDO10|      13.65|         Regular|
|          FDP10|       NULL|         Low Fat|
|          FDH17|       16.2|         Regular|
|          FDU28|       19.2|         Regular|
|          FDY07|       11.8|         Low Fat|
|          FDA03|       18.5|         Regular|
|          FDX32|       15.1|         Regular|
|          FDS46|       17.6|         Regular|
|          FDF32|      16.35|         Low Fat|
|          FDP49|        9.0|         Regular|
|          NCB42|       11.8|         Low Fat|
|          FDP49|        9.0|         Regular|
|          DR

### COL

In [42]:
df.select(col('Item_Identifier'),col('Item_Weight'),col('Item_Fat_Content')).show()

+---------------+-----------+----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|
+---------------+-----------+----------------+
|          FDA15|        9.3|         Low Fat|
|          DRC01|       5.92|         Regular|
|          FDN15|       17.5|         Low Fat|
|          FDX07|       19.2|         Regular|
|          NCD19|       8.93|         Low Fat|
|          FDP36|     10.395|         Regular|
|          FDO10|      13.65|         Regular|
|          FDP10|       NULL|         Low Fat|
|          FDH17|       16.2|         Regular|
|          FDU28|       19.2|         Regular|
|          FDY07|       11.8|         Low Fat|
|          FDA03|       18.5|         Regular|
|          FDX32|       15.1|         Regular|
|          FDS46|       17.6|         Regular|
|          FDF32|      16.35|         Low Fat|
|          FDP49|        9.0|         Regular|
|          NCB42|       11.8|         Low Fat|
|          FDP49|        9.0|         Regular|
|          DR

### Alias - col is mandatory for alias

In [45]:
df.select(col('Item_Identifier')\
          .alias('Item_ID'))\
          .show()

+-------+
|Item_ID|
+-------+
|  FDA15|
|  DRC01|
|  FDN15|
|  FDX07|
|  NCD19|
|  FDP36|
|  FDO10|
|  FDP10|
|  FDH17|
|  FDU28|
|  FDY07|
|  FDA03|
|  FDX32|
|  FDS46|
|  FDF32|
|  FDP49|
|  NCB42|
|  FDP49|
|  DRI11|
|  FDU02|
+-------+
only showing top 20 rows



### FILTER/WHERE

1. Filter data with fat content=Regular
2. Slice the data with item type=Soft Drinks and weight<10
3. Fetch the data with Tier in (Tier1 or Tier2) and outler size is null

In [5]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

In [8]:
#scenario 1

df.filter(col('Item_Fat_Content')=='Regular').show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228|
|          FDX07|       19.2|         Regular|            0.0|Fruits and Vegeta...| 182.095|           OUT010|                     1998|       NULL|              Tier 3|    Gro

In [ ]:
#Scenario-2
df.filter( (col('Item_Type')=='Soft Drinks') & \
            (col('Item_Weight')<10))\
            .show()

In [ ]:
#Scenario-3
df.filter(
    (col('Outlet_Size').isNull()) &
    (col('Outlet_Location_Type').isin('Tier 1','Tier 2'))
).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDH17|       16.2|         Regular|    0.016687114|        Frozen Foods| 96.9726|           OUT045|                     2002|       NULL|              Tier 2|Supermarket Type1|        1076.5986|
|          FDU28|       19.2|         Regular|     0.09444959|        Frozen Foods|187.8214|           OUT017|                     2007|       NULL|              Tier 2|Superma

### withColumnRenamed

In [16]:
df.withColumnRenamed('Item_Weight','Item_Wt').show()

+---------------+-------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Wt|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|    9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|   5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         

### withColumns

1. Add New Column
2. Modify Existing Column

In [19]:
#Scenario 1

df=df.withColumn('flag',lit("new"))
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

In [20]:
#Scenario 2
df.withColumn('multiply',col('item_Weight')*col('Item_MRP')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|          multiply|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|2323.2255600000003|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drin

Replace the values on exisitng col

In [45]:
df.withColumn('Item_Fat_Content',\
              regexp_replace(col('Item_Fat_Content'),"Regular","Reg"))\
              .withColumn('Item_Fat_Content',\
                    regexp_replace(col('Item_Fat_Content'),"Low Fat","LF")).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|              LF|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|             Reg|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

### Type Casting

In [46]:
df=df.withColumn('Item_Weight',col('Item_Weight').cast(StringType()))
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)
 |-- flag: string (nullable = false)



### Sort/orderBy - Default is Asc

1. sort by Item Weight in desc
2. sort by Item_Visibility in Asc
3. sort by Item_weight and Item_Visibility in Desc

In [47]:
df.sort(col('Item_Weight').desc()).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDR13|      9.895|         Regular|    0.028696932|      Canned|117.0492|           OUT013|                     1987|       High|              Tier 3|Supermarket Type1|         810.9444| new|
|          DRD49|      9.895|         Low Fat|    0.167799329| Soft Drinks|239.4564|           OUT035|                     2004|      Small|              Tier 2|Supermarket Type1|        5

In [48]:
df.sort(col('Item_Visibility').asc()).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          DRB48|      16.75|         Regular|            0.0|         Soft Drinks| 39.3822|           OUT046|                     1997|      Small|              Tier 1|Supermarket Type1|         353.5398| new|
|          FDS32|      17.75|         Regular|            0.0|Fruits and Vegeta...|139.9838|           OUT045|                     2002|       NULL|        

In [33]:
df.sort(['Item_Weight','Item_Visibility'],ascending=[0,0]).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          DRD49|      9.895|         Low Fat|    0.168780385|         Soft Drinks|236.8564|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|         4767.128| new|
|          DRD49|      9.895|         Low Fat|     0.16817143|         Soft Drinks|237.7564|           OUT045|                     2002|       NULL|        

In [34]:
df.sort(['Item_Weight','Item_Visibility'],ascending=[0,1]).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDR13|      9.895|         Regular|    0.028696932|              Canned|117.0492|           OUT013|                     1987|       High|              Tier 3|Supermarket Type1|         810.9444| new|
|          FDR13|      9.895|         Regular|    0.028765486|              Canned|115.3492|           OUT049|                     1999|     Medium|        

### Limit

In [ ]:
df.limit(5).show()


+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

### DROP

In [37]:
df.drop('Item_Visibility').show()

+---------------+-----------+----------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|         Low Fat|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|         Regular|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228| new|
|          FDN15|       1

In [38]:
df.drop('Item_Visibility','Item_Type').show()

+---------------+-----------+----------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|         Low Fat|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|         Regular| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228| new|
|          FDN15|       17.5|         Low Fat| 141.618|           OUT049|                     1999|     Medium|              Tier 

### DROP_DUPLICATES

In [42]:
df.dropDuplicates().show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          NCW29|       14.0|         Low Fat|    0.028907832|  Health and Hygiene| 130.431|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|          778.986| new|
|          DRJ37|       10.8|         Low Fat|    0.061101383|         Soft Drinks|151.9024|           OUT046|                     1997|      Small|        

In [44]:
df.drop_duplicates(subset=['Item_Type']).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDP36|     10.395|         Regular|            0.0|        Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088| new|
|          FDO23|      17.85|         Low Fat|            0.0|              Breads| 93.1436|           OUT045|                     2002|       NULL|        

In [50]:
df.distinct().show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          NCW29|       14.0|         Low Fat|    0.028907832|  Health and Hygiene| 130.431|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|          778.986| new|
|          DRJ37|       10.8|         Low Fat|    0.061101383|         Soft Drinks|151.9024|           OUT046|                     1997|      Small|        

### UNION AND UNION BYNAME

In [53]:
data1=[('1','kad'),
       ('2','sid')]
schema1='id STRING,name STRING'

df1=spark.createDataFrame(data1,schema1)

data2=[('3','rahul'),
       ('4','jas')]
schema2='id STRING,name STRING'

df2=spark.createDataFrame(data2,schema2)




In [54]:
df1.show()

+---+----+
| id|name|
+---+----+
|  1| kad|
|  2| sid|
+---+----+



In [55]:
df2.show()

+---+-----+
| id| name|
+---+-----+
|  3|rahul|
|  4|  jas|
+---+-----+



UNION

In [56]:
df1.union(df2).show()

+---+-----+
| id| name|
+---+-----+
|  1|  kad|
|  2|  sid|
|  3|rahul|
|  4|  jas|
+---+-----+



In [58]:
data1=[('kad','1'),
       ('sid','2')]
schema1='name STRING,id STRING'

df1=spark.createDataFrame(data1,schema1)
df1.show()

+----+---+
|name| id|
+----+---+
| kad|  1|
| sid|  2|
+----+---+



In [59]:
df1.union(df2).show()

+----+-----+
|name|   id|
+----+-----+
| kad|    1|
| sid|    2|
|   3|rahul|
|   4|  jas|
+----+-----+



UNION BY NAME

In [60]:
df1.unionByName(df2).show()

+-----+---+
| name| id|
+-----+---+
|  kad|  1|
|  sid|  2|
|rahul|  3|
|  jas|  4|
+-----+---+



### STRING FUNCTIONS

INITCAP()

UPPER()

LOWER()

In [5]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

Initcap()

In [6]:
df.select(initcap('Item_Type')).show(truncate=False)

+---------------------+
|initcap(Item_Type)   |
+---------------------+
|Dairy                |
|Soft Drinks          |
|Meat                 |
|Fruits And Vegetables|
|Household            |
|Baking Goods         |
|Snack Foods          |
|Snack Foods          |
|Frozen Foods         |
|Frozen Foods         |
|Fruits And Vegetables|
|Dairy                |
|Fruits And Vegetables|
|Snack Foods          |
|Fruits And Vegetables|
|Breakfast            |
|Health And Hygiene   |
|Breakfast            |
|Hard Drinks          |
|Dairy                |
+---------------------+
only showing top 20 rows



upper() and lower()

In [7]:
df.select(upper('Item_Type').alias('Upper_Item_Type')).show(truncate=False)
df.select(lower('Item_Type')).show(truncate=False)

+---------------------+
|Upper_Item_Type      |
+---------------------+
|DAIRY                |
|SOFT DRINKS          |
|MEAT                 |
|FRUITS AND VEGETABLES|
|HOUSEHOLD            |
|BAKING GOODS         |
|SNACK FOODS          |
|SNACK FOODS          |
|FROZEN FOODS         |
|FROZEN FOODS         |
|FRUITS AND VEGETABLES|
|DAIRY                |
|FRUITS AND VEGETABLES|
|SNACK FOODS          |
|FRUITS AND VEGETABLES|
|BREAKFAST            |
|HEALTH AND HYGIENE   |
|BREAKFAST            |
|HARD DRINKS          |
|DAIRY                |
+---------------------+
only showing top 20 rows

+---------------------+
|lower(Item_Type)     |
+---------------------+
|dairy                |
|soft drinks          |
|meat                 |
|fruits and vegetables|
|household            |
|baking goods         |
|snack foods          |
|snack foods          |
|frozen foods         |
|frozen foods         |
|fruits and vegetables|
|dairy                |
|fruits and vegetables|
|snack foods  

### DATE FUNCTIONS

CURRENT_DATE()
DATE_ADD()
DATE_SUB()

Current_Date

In [44]:
df=spark.read.format('csv')\
            .option('inferSchema',True)\
            .option('header',True)\
            .load('BigMart_Sales.csv')

In [45]:
df=df.withColumn('curr_date',current_date())
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2

Date_Add()

In [46]:
from pyspark.sql.functions import date_add
df_add=df.withColumn('week_after',date_add('curr_date',7))
df_add.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2

In [47]:
df=df_add.withColumn('week_before',date_sub('curr_date',7))
#you can also do same with -7 in date add
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 2025-03-29|
|          DRC01|       5.92|         Re

### DATEDIFF

In [48]:
df=df.withColumn('date_diff',date_diff('curr_date','week_after'))
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 2025-03-29|       -7|


### Date_Format

In [49]:

df=df.withColumn('week_before',date_format('week_before','dd-MM-yyyy'))
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|       -7|


### Handling NULLS

1. Dropping Nulls
2. Filling Nulls

Dropping Nulls

In [55]:
df.dropna('all').show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|       -7|


In [57]:
df.dropna('any').show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|       -7|


In [59]:
df.dropna(subset='Outlet_Size').show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|       -7|


Filling Nulls

In [61]:
df.fillna('#NotAvailable').show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|  Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|       Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|   

In [63]:
df.fillna('NotAvailable',subset=['Outlet_Size']).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+------------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year| Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales| curr_date|week_after|week_before|date_diff|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+------------+--------------------+-----------------+-----------------+----------+----------+-----------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|      Medium|              Tier 1|Supermarket Type1|         3735.138|2025-04-05|2025-04-12| 29-03-2025|       

### SPLIT AND Indexing

In [4]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

In [5]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|         Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|[Supermarket, Type1]|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Ti

In [6]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')[1]).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|      Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|      Type2|         443.4228|


### EXPLODE

In [7]:
df_exp=df.withColumn('Outlet_Type',split('Outlet_Type',' '))

df_exp.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|         Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|[Supermarket, Type1]|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Ti

In [5]:
df_exp.withColumn('Outlet_Type',explode('Outlet_Type')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket|         3735.138|
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|      Type1|         3735.138|


### ARRAY_CONTAINS

In [8]:
df_exp.withColumn('Type1_flg',array_contains('Outlet_Type','Type1')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|         Outlet_Type|Item_Outlet_Sales|Type1_flg|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+---------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|[Supermarket, Type1]|         3735.138|     true|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|              

### GROUP_BY

In [10]:
df.groupBy('Item_Type').agg(sum('Item_MRP')).show()

+--------------------+------------------+
|           Item_Type|     sum(Item_MRP)|
+--------------------+------------------+
|       Starchy Foods|21880.027399999995|
|        Baking Goods| 81894.73640000001|
|              Breads| 35379.11979999999|
|Fruits and Vegeta...|178124.08099999998|
|                Meat|59449.863799999956|
|         Hard Drinks|29334.676599999995|
|         Soft Drinks|58514.164999999964|
|           Household|135976.52539999998|
|           Breakfast|        15596.6966|
|               Dairy|101276.45959999996|
|         Snack Foods|175433.92040000003|
|              Others|22451.891600000006|
|             Seafood| 9077.870000000003|
|              Canned|  90706.7269999999|
|        Frozen Foods|118558.88140000001|
|  Health and Hygiene|        68025.8388|
+--------------------+------------------+



In [12]:
df.groupBy('Item_Type').agg(avg('Item_MRP').alias('AVG')).show()

+--------------------+------------------+
|           Item_Type|               AVG|
+--------------------+------------------+
|       Starchy Foods|147.83802297297294|
|        Baking Goods|126.38076604938273|
|              Breads| 140.9526685258964|
|Fruits and Vegeta...|144.58123457792206|
|                Meat|139.88203247058814|
|         Hard Drinks|137.07792803738315|
|         Soft Drinks|131.49250561797746|
|           Household|149.42475318681318|
|           Breakfast|141.78815090909092|
|               Dairy|148.49920762463336|
|         Snack Foods|146.19493366666669|
|              Others|132.85142958579885|
|             Seafood|141.84171875000004|
|              Canned|139.76383204930647|
|        Frozen Foods|138.50336612149533|
|  Health and Hygiene|130.81892076923077|
+--------------------+------------------+



In [15]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP').alias('SUM'),avg('Item_MRP').alias("AVG")).show()

+--------------------+-----------+------------------+------------------+
|           Item_Type|Outlet_Size|               SUM|               AVG|
+--------------------+-----------+------------------+------------------+
|       Starchy Foods|     Medium| 7124.136199999997| 148.4195041666666|
|Fruits and Vegeta...|     Medium|59047.217200000014| 142.9714702179177|
|       Starchy Foods|       NULL|         6040.6402|140.48000465116277|
|              Breads|       NULL|        10011.5004|139.04861666666667|
|        Baking Goods|       NULL|23433.838799999994|126.66939891891889|
|Fruits and Vegeta...|       NULL|49758.730999999985|142.57516045845267|
|        Frozen Foods|       High|12588.291000000001|         136.82925|
|         Soft Drinks|       High| 6456.165199999999|131.75847346938772|
|           Breakfast|      Small|3917.0407999999998|130.56802666666667|
|                Meat|     Medium| 20326.45059999999|136.41913154362408|
|Fruits and Vegeta...|       High| 20671.3475999999

### COLLECT_LIST

In [18]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema='user string, book string'

df_book=spark.createDataFrame(data,schema)

df_book.show()

+-----+-----+
| user| book|
+-----+-----+
|user1|book1|
|user1|book2|
|user2|book2|
|user2|book4|
|user3|book1|
+-----+-----+



In [19]:
df_book.groupBy('user').agg(collect_list('book')).show()

+-----+------------------+
| user|collect_list(book)|
+-----+------------------+
|user1|    [book1, book2]|
|user2|    [book2, book4]|
|user3|           [book1]|
+-----+------------------+



### PIVOT

In [20]:
df.groupby('item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).show()

+--------------------+------------------+------------------+------------------+------------------+
|           item_Type|              null|              High|            Medium|             Small|
+--------------------+------------------+------------------+------------------+------------------+
|       Starchy Foods|140.48000465116277|158.15707368421053| 148.4195041666666| 150.2701736842105|
|              Breads|139.04861666666667|         133.75896| 140.8610385542169| 145.5236507042254|
|        Baking Goods|126.66939891891889|129.20204383561642|126.17856847290639|125.21336363636368|
|Fruits and Vegeta...|142.57516045845267|145.57287042253515| 142.9714702179177|148.31336951219507|
|                Meat|139.29453448275865| 137.2447902439025|136.41913154362408|145.69925042016808|
|         Hard Drinks| 134.3875333333333| 141.9275217391304|142.83769599999994|        129.758784|
|         Soft Drinks|133.42344360902257|131.75847346938772| 128.2696817518248| 132.8550428571429|
|         

### WHEN-OTHERWISE

Scenario - 1 - if item_type is meat is non-veg else veg

In [24]:
df=df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('veg'))

Scenario - 2 - if item type is veg and item mrp is <100 then it is inexpensive else veg expensive otherwise non veg

In [29]:
df.withColumn('veg_exp_flg',when(((col('veg_flag')=='veg') \
              & (col('Item_MRP')<100)),'Veg_Inexpensive') \
              .when((col('veg_flag')=='veg') \
              & (col('Item_MRP')>100),'Veg_Expensive') \
              .otherwise('Non_Veg')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+---------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|    veg_exp_flg|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+---------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|     veg|  Veg_Expensive|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft 

### JOINS

- Inner Join
- Left Join
- Right Join
- Full Join
- Anti Join

In [30]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING' 

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [31]:
df1.show()
df2.show()

+------+--------+-------+
|emp_id|emp_name|dept_id|
+------+--------+-------+
|     1|    gaur|    d01|
|     2|     kit|    d02|
|     3|     sam|    d03|
|     4|     tim|    d03|
|     5|    aman|    d05|
|     6|     nad|    d06|
+------+--------+-------+

+-------+----------+
|dept_id|department|
+-------+----------+
|    d01|        HR|
|    d02| Marketing|
|    d03|  Accounts|
|    d04|        IT|
|    d05|   Finance|
+-------+----------+



INNER JOIN

In [32]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     3|     sam|    d03|    d03|  Accounts|
|     4|     tim|    d03|    d03|  Accounts|
|     5|    aman|    d05|    d05|   Finance|
+------+--------+-------+-------+----------+



LEFT JOIN

In [37]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'LEFT').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     3|     sam|    d03|    d03|  Accounts|
|     4|     tim|    d03|    d03|  Accounts|
|     5|    aman|    d05|    d05|   Finance|
|     6|     nad|    d06|   NULL|      NULL|
+------+--------+-------+-------+----------+



RIGHT JOIN

In [36]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'right').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     4|     tim|    d03|    d03|  Accounts|
|     3|     sam|    d03|    d03|  Accounts|
|  NULL|    NULL|   NULL|    d04|        IT|
|     5|    aman|    d05|    d05|   Finance|
+------+--------+-------+-------+----------+



ANTI JOIN

In [38]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'anti').show()

+------+--------+-------+
|emp_id|emp_name|dept_id|
+------+--------+-------+
|     6|     nad|    d06|
+------+--------+-------+



### WINDOW FUNCTIONS

- ROW NUMBER()
- RANK()
- DENSE_RANK()

ROW_NUMBER()

In [39]:
from pyspark.sql.window import Window

In [40]:
df.withColumn('rowCol',row_number().over(Window.orderBy('Item_Identifier'))).show()

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|rowCol|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------+
|          DRA12|       11.6|         Low Fat|    0.041177505|Soft Drinks|140.3154|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|        2552.6772|     veg|     1|
|          DRA12|       11.6|         Low Fat|            0.0|Soft Drinks|141.6154|           OUT045|                     2002|       NULL|         

RANK()

In [42]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc()))).show()

+---------------+-----------+----------------+---------------+------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|         Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|rank|
+---------------+-----------+----------------+---------------+------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+----+
|          NCZ54|      14.65|         Low Fat|    0.083359391|         Household|161.9552|           OUT046|                     1997|      Small|              Tier 1|Supermarket Type1|        4711.2008|     veg|   1|
|          NCZ54|       NULL|         Low Fat|    0.082955719|         Household|164.0552|           OUT027|                    

Dense_Rank()

In [43]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc()))) \
    .withColumn('denserank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).show()

+---------------+-----------+----------------+---------------+------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+----+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|         Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|rank|denserank|
+---------------+-----------+----------------+---------------+------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+----+---------+
|          NCZ54|      14.65|         Low Fat|    0.083359391|         Household|161.9552|           OUT046|                     1997|      Small|              Tier 1|Supermarket Type1|        4711.2008|     veg|   1|        1|
|          NCZ54|       NULL|         Low Fat|    0.082955719|         Household|164.055

### WINDOW FUNCTIONS

- Cumulative Sum

In [44]:
df.withColumn('cum_sum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|          cum_sum|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+-----------------+
|          FDP36|     10.395|         Regular|            0.0|Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088|     veg|81894.73640000001|
|          FDW12|       NULL|         Regular|    0.035399923|Baking Goods|144.5444|           OUT02

In [45]:
df.withColumn('cum_sum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|           cum_sum|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|          FDP36|     10.395|         Regular|            0.0|Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088|     veg|           51.4008|
|          FDW12|       NULL|         Regular|    0.035399923|Baking Goods|144.5444|           O

In [47]:
df.withColumn('total_sum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|         total_sum|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|          FDP36|     10.395|         Regular|            0.0|Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088|     veg|1201681.4808000019|
|          FDW12|       NULL|         Regular|    0.035399923|Baking Goods|144.5444|           O

### USER DEFINED FUNCTIONS

- create UDF
- create python function using udf()

In [48]:
def my_func(x):
    return x*x

In [49]:
my_udf=udf(my_func)

In [50]:
df.withColumn('New_Col',my_udf('Item_MRP')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|           New_Col|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|     veg|62404.636404640005|
|          DRC01|       5.92|         Regular|    0.019278216|  

### DATA WRITING

- CSV

In [59]:
df.write.format('csv')\
    .mode('overwrite')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

###DATA WRITING MODES

- APPEND - TO APPEND DATA
- OVERWRITE - TO OVERWRITE DATA
- ERROR - To Throw an error if file is already exists
- IGNORE - To IGNORE any error

In [60]:
df.write.format('csv')\
    .mode('append')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

In [61]:
df.write.format('csv')\
    .mode('overwrite')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

In [62]:
df.write.format('csv')\
    .mode('error')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/D:/Data Engineering/Pyspark/Ansh L/write already exists. Set mode as "overwrite" to overwrite the existing path.

In [63]:
df.write.format('csv')\
    .mode('Ignore')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

### PARQUET FILE FORMAT

- ROW BASED FILE FORMAT

In [64]:
df.write.format('parquet')\
    .mode('overwrite')\
        .save('D:/Data Engineering/Pyspark/Ansh L/write/')

### Table

In [65]:
df.write.format('parquet')\
    .mode('overwrite')\
        .saveAsTable('my_table')

### SPARK SQL

In [66]:
df.createTempView('my_view')

In [72]:
data2=spark.sql("""
select * from my_view""")

In [73]:
data2.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|     veg|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|    

In [78]:
spark.sql("""
select * from my_view where Item_fat_content='Low Fat'
                """).show()
#data3.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|veg_flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+--------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|     veg|
|          FDN15|       17.5|         Low Fat|    0.016760075|                Meat| 141.618|           OUT049|                     1999|    